# Real City Case (Multiprocessing)

※Warning: Multiple GPUs are required to run this code!

※Warning: Real data is not uploaded. Only few example data is uploaded. Just see how simulation goes. 

This notebook gives a brief overview on how to train multiple configs using multiprocessing.

In [1]:
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
import sys 
sys.path.append("../")
import osmnx as ox
import torch as th
import networkx as nx
import dgl

Using backend: pytorch


In [2]:
DATA_FOLDER='../data_example/'

Here is a common parameters example.

In [3]:
TRAIN_EPOCH = 5
TRAIN_TIME_STEPS = 1438 

common_parameters = {
    "graph_data": DATA_FOLDER+'seoul_rectangular_drive_network_simplified_with_speed.graphml',
    
    "call_generator_data": DATA_FOLDER+'seoul_call_data_20181023.csv',
    "driver_initializer_data": DATA_FOLDER+'seoul_idle_driver_initial_distribution_20181023.csv',
    "total_driver_number_per_time_data": DATA_FOLDER+'seoul_total_driver_per_time_20181023.csv',
    "speed_info_data": DATA_FOLDER+'.road_speed_seoul_20181023.csv',
    
    "name":'seoul_with_speed',
    "consider_speed": True,
    "epochs": TRAIN_EPOCH, 
    "time_steps": TRAIN_TIME_STEPS,
    "log_save_folder": 'train_log_multiprocessing',
    "model_save_folder": 'model_data_multiprocessing',
    "write_log": True,
    "save_model": True,
    "verbos": False
}

Here is a config example with different settings.

In [4]:
from main import *

kwargs_list = []

# (0) Simple (not required if train only)
# kwargs_list.append({"model_type":'random'})
# kwargs_list.append({"model_type":'proportional', "strategy": 1, "policy_pow": 1.0})

# (1) Argmax
kwargs_list.append({"model_type":'gcn', "strategy": POLICY_ARGMAX, "gpu_id":0, "epsilon_min":0.1})
kwargs_list.append({"model_type":'gat', "strategy": POLICY_ARGMAX, "gpu_id":0, "epsilon_min":0.1})

# (2) Pow
kwargs_list.append({"model_type":'gcn', "strategy": POLICY_POW, "gpu_id":1, "policy_pow":3})
kwargs_list.append({"model_type":'gat', "strategy": POLICY_POW, "gpu_id":1, "policy_pow":3})

# (3) Exp
kwargs_list.append({"model_type":'gcn', "strategy": POLICY_EXP, "gpu_id":2, "temperature":0.02})
kwargs_list.append({"model_type":'gat', "strategy": POLICY_EXP, "gpu_id":2, "temperature":0.02})

# (4) Entropy Softmax
kwargs_list.append({"model_type":'gcn', "strategy": POLICY_ENTROPY, "gpu_id":3, "temperature":0.02})
kwargs_list.append({"model_type":'gat', "strategy": POLICY_ENTROPY, "gpu_id":3, "temperature":0.02})

In [5]:
# may be changed
common_parameters['driver_coefficient'] = 0.5

Finally run train and evaluate using multiprocessing. See code in `main.py` for details.

In [ ]:
train_using_multiprocessing(common_parameters, kwargs_list)
evaluate_using_multiprocessing(common_parameters, kwargs_list)